##############################################################################################################################
### Configure the environment
##############################################################################################################################

In [3]:
# Section must be included at the beginning of each new notebook. Remember to change the app name. 
# If you're using VirtualBox, change the below to '/home/user/spark-2.1.1-bin-hadoop2.7'
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('missing').getOrCreate()

In [4]:
# Other environment imports
from pyspark.sql.functions import datediff, to_date, lit, when, date_sub, col
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

##############################################################################################################################
### Get and inspect the data
##############################################################################################################################

In [5]:
# Get the data files
standings1617 = spark.read.csv("2016-17_standings.csv", header=True, inferSchema=True)
standings1617.show(5)

standings1718 = spark.read.csv("2017-18_standings.csv", header=True, inferSchema=True)
standings1718.show(5)

boxScore1617 = spark.read.csv("2016-17_teamBoxScore.csv", header=True, inferSchema=True)
boxScore1617.show(5)

boxScore1718 = spark.read.csv("2017-18_teamBoxScore.csv", header=True, inferSchema=True)
boxScore1718.show(5)
boxScore1718.head(5)


+--------------------+--------+----+-------+-------+--------+---+-------+------+--------+------+--------+-------+--------+-------+--------+-------+--------+--------+-------+--------+--------+--------+-------+----------+---------+--------------+-------------+---+-----------+---+---+------+------------+---------+---------+-----------+--------+--------+
|              stDate|teamAbbr|rank|rankOrd|gameWon|gameLost|stk|stkType|stkTot|gameBack|ptsFor|ptsAgnst|homeWin|homeLoss|awayWin|awayLoss|confWin|confLoss|lastFive|lastTen|gamePlay|ptsScore|ptsAllow|ptsDiff|opptGmPlay|opptGmWon|opptOpptGmPlay|opptOpptGmWon|sos|relPercIndx|mov|srs|pwPerc|pythPerc1391|wpyth1391|lpyth1391|pythPerc165|wpyth165|lpyth165|
+--------------------+--------+----+-------+-------+--------+---+-------+------+--------+------+--------+-------+--------+-------+--------+-------+--------+--------+-------+--------+--------+--------+-------+----------+---------+--------------+-------------+---+-----------+---+---+------+-----

+--------------------+------+-------+-------+-------+---------+-------+---------+-------+--------+--------+---------+-------+--------+-------+----------+-------+-------+------+-------+-------+------+-------+-------+----------+-------+-------+----------+-------+-------+----------+-------+-------+----------+-------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+------------+------------+----------+-----------+------------+------------+----------+-----------+-----------+--------+-------+-------+---------+--------+--------+---------+------------+-------+---------+---------+--------+--------+---------+-------+--------+-------+----------+-------+-------+------+-------+-------+------+-------+-------+----------+-------+-------+----------+-------+-------+----------+-------+-------+----------+-------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+------------+------------+----------+-----------+------------+-------

[Row(gmDate=datetime.datetime(2017, 10, 17, 0, 0), gmTime='08:00', seasTyp='Regular', offLNm1='Forte', offFNm1='Brian', offLNm2='Smith', offFNm2='Michael', offLNm3='McCutchen', offFNm3='Monty', teamAbbr='BOS', teamConf='East', teamDiv='Atlantic', teamLoc='Away', teamRslt='Loss', teamMin=241, teamDayOff=0, teamPTS=99, teamAST=24, teamTO=12, teamSTL=11, teamBLK=4, teamPF=24, teamFGA=88, teamFGM=36, teamFGPerc=0.4091, team2PA=56, team2PM=28, team2PPerc=0.5, team3PA=32, team3PM=8, team3PPerc=0.25, teamFTA=25, teamFTM=19, teamFTPerc=0.76, teamORB=9, teamDRB=37, teamTRB=46, teamPTS1=19, teamPTS2=19, teamPTS3=33, teamPTS4=28, teamPTS5=0, teamPTS6=0, teamPTS7=0, teamPTS8=0, teamTREBPerc=47.9167, teamASSTPerc=66.6667, teamTSPerc=0.5, teamEFGPerc=0.4545, teamOREBPerc=18.0, teamDREBPerc=80.4348, teamTOPerc=10.8108, teamSTLPerc=10.9692, teamBLKPerc=3.9888, teamBLKR=7.1429, teamPPS=1.125, teamFIC=75.375, teamFIC40=62.5519, teamOrtg=98.7227, teamDrtg=101.7143, teamEDiff=-2.9916, teamPlayPerc=0.3956,

##############################################################################################################################
### For the Box Score preparation - the next set of steps are for the box score data preparation
##############################################################################################################################

In [6]:
# Create column for the "days since" the start of the season for 2016/17

boxScore1617 = boxScore1617.withColumn('daysSince', datediff(to_date(boxScore1617['gmDate']), to_date(lit('2016-10-25'))))
boxScore1617.show(5)

+--------------------+------+-------+-------+-------+--------+-------+--------+-------+--------+--------+---------+-------+--------+-------+----------+-------+-------+------+-------+-------+------+-------+-------+----------+-------+-------+----------+-------+-------+----------+-------+-------+----------+-------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+------------+------------+----------+-----------+------------+------------+----------+-----------+-----------+--------+-------+-------+---------+--------+--------+---------+------------+-------+---------+---------+--------+--------+---------+-------+--------+-------+----------+-------+-------+------+-------+-------+------+-------+-------+----------+-------+-------+----------+-------+-------+----------+-------+-------+----------+-------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+------------+------------+----------+-----------+------------+---------

In [7]:
# Create column for the "days since" the start of the season for 2017/18

boxScore1718 = boxScore1718.withColumn('daysSince', datediff(to_date(boxScore1718['gmDate']), to_date(lit('2016-10-17'))))
boxScore1718.show(5)

+--------------------+------+-------+-------+-------+---------+-------+---------+-------+--------+--------+---------+-------+--------+-------+----------+-------+-------+------+-------+-------+------+-------+-------+----------+-------+-------+----------+-------+-------+----------+-------+-------+----------+-------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+------------+------------+----------+-----------+------------+------------+----------+-----------+-----------+--------+-------+-------+---------+--------+--------+---------+------------+-------+---------+---------+--------+--------+---------+-------+--------+-------+----------+-------+-------+------+-------+-------+------+-------+-------+----------+-------+-------+----------+-------+-------+----------+-------+-------+----------+-------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+------------+------------+----------+-----------+------------+-------

In [8]:
# Merge the two years dataframes of Games Played data
boxScoreAll = boxScore1617.union(boxScore1718)

print("Box Score 1617:", boxScore1617.count())
print("Box Score 1718:", boxScore1718.count())
print("Box Score All:", boxScoreAll.count())

Box Score 1617: 2460
Box Score 1718: 2460
Box Score All: 4920


In [9]:
# Remove all the "Away" perspective records by selecting only the "Home" records
boxScoreAll = boxScoreAll.filter('teamLoc = "Home"')
print("Box Score All:", boxScoreAll.count())

Box Score All: 2460


In [ ]:
# Visualize the Outliers

myBS = pd.DataFrame(boxScoreAll.take(5000))

plotData = pd.DataFrame(myBS, columns = [13, 16]).rename(inplace=False, index = str, columns = {13:"teamRslt",16:'teamPTS'}).pivot(columns='teamRslt')
print(plotData)

fig, axes = plt.subplots(nrows=1, ncols=2, sharey=True)
axes[0].boxplot(plotData, showfliers = True, labels=list('Win', 'Lose'))
axes[0].set_title("Scores by Result for Home team")

plotData = pd.DataFrame(myBS, columns = [69, 72]).rename(inplace=False, index = str, columns = {69:"teamRslt",72:'teamPTS'})

axes[1].boxplot(plotData, showfliers = True, labels=list('Win', 'Lose'))
axes[1].set_title("Scores by Result for Away team")

fig.subplots_adjust(hspace=50)
plt.show()

In [71]:
# Remove the columns that are not known before the game starts by selecting the columns that do.
boxScoreAll = boxScoreAll.select('gmDate','gmTime','seasTyp','offLNm1','offFNm1','offLNm2','offFNm2','offLNm3','offFNm3','teamAbbr','teamConf','teamDiv','teamLoc','teamRslt','teamDayOff','opptAbbr','opptConf','opptDiv','opptLoc','opptDayOff','daysSince')
boxScoreAll.show(5)

+--------------------+------+-------+--------+-------+--------+-------+----------+-------+--------+--------+---------+-------+--------+----------+--------+--------+---------+-------+----------+---------+
|              gmDate|gmTime|seasTyp| offLNm1|offFNm1| offLNm2|offFNm2|   offLNm3|offFNm3|teamAbbr|teamConf|  teamDiv|teamLoc|teamRslt|teamDayOff|opptAbbr|opptConf|  opptDiv|opptLoc|opptDayOff|daysSince|
+--------------------+------+-------+--------+-------+--------+-------+----------+-------+--------+--------+---------+-------+--------+----------+--------+--------+---------+-------+----------+---------+
|2016-10-25 00:00:...| 08:00|Regular|    Lane|   Karl|   Adams| Bennie|   Kennedy|   Bill|     CLE|    East|  Central|   Home|     Win|         0|      NY|    East| Atlantic|   Away|         0|        0|
|2016-10-25 00:00:...| 10:00|Regular| Buchert|   Nick|Callahan|   Mike|     Brown|   Tony|     POR|    West|Northwest|   Home|     Win|         0|     UTA|    West|Northwest|   Away|  

In [72]:
# Fix the rows that have the wrong times - they need 12hr deducted from their times
boxScoreAll.filter('gmTime > "12:30"').select('gmtime').show()

boxScoreAll = boxScoreAll.withColumn("gmTime", when(boxScoreAll["gmTime"] > '12:30', boxScoreAll["gmTime"] - '12:00').otherwise(boxScoreAll["gmTime"]))

boxScoreAll.filter('gmTime > "12:30"').select('gmtime').show()

+------+
|gmtime|
+------+
| 18:00|
| 18:00|
| 19:00|
| 19:00|
+------+

+------+
|gmtime|
+------+
+------+



##############################################################################################################################
### For the Standings preparation - the next set of steps are for the standings data preparation
##############################################################################################################################

In [73]:
# Join the two years datasets
standingsAll = standings1617.union(standings1718)
standingsAll.show(1)

# Rename the 'teamAbbr' column to prevent clash later
standingsAll = standingsAll.withColumnRenamed('teamAbbr', 'steamAbbr')
standingsAll.show(1)

print("Standings 1617:", standings1617.count())
print("Standings 1718:", standings1718.count()) 
# There are more standings rows in the 17/18 dataset because the season runs for a week longer
print("Standings All:", standingsAll.count())

+--------------------+--------+----+-------+-------+--------+---+-------+------+--------+------+--------+-------+--------+-------+--------+-------+--------+--------+-------+--------+--------+--------+-------+----------+---------+--------------+-------------+---+-----------+---+---+------+------------+---------+---------+-----------+--------+--------+
|              stDate|teamAbbr|rank|rankOrd|gameWon|gameLost|stk|stkType|stkTot|gameBack|ptsFor|ptsAgnst|homeWin|homeLoss|awayWin|awayLoss|confWin|confLoss|lastFive|lastTen|gamePlay|ptsScore|ptsAllow|ptsDiff|opptGmPlay|opptGmWon|opptOpptGmPlay|opptOpptGmWon|sos|relPercIndx|mov|srs|pwPerc|pythPerc1391|wpyth1391|lpyth1391|pythPerc165|wpyth165|lpyth165|
+--------------------+--------+----+-------+-------+--------+---+-------+------+--------+------+--------+-------+--------+-------+--------+-------+--------+--------+-------+--------+--------+--------+-------+----------+---------+--------------+-------------+---+-----------+---+---+------+-----

##############################################################################################################################
### Merge the standings data into the box score data
##############################################################################################################################

In [74]:
# Home team data
boxWithStand = boxScoreAll.join(standingsAll, (boxScoreAll.teamAbbr == standingsAll.steamAbbr) & (date_sub(boxScoreAll.gmDate, 1) == to_date(standingsAll.stDate)))
print(boxScoreAll.count())
print(boxWithStand.count())

2460
2460


In [76]:
# Away team data

# Rename columns by adding a suffix so that there are no name clashes
opptStandings = standingsAll.select(*(col(x).alias(x + '_oppt') for x in standingsAll.columns))

# Join the tables
boxWithStand = boxWithStand.join(opptStandings, (boxScoreAll.opptAbbr == opptStandings.steamAbbr_oppt) & (date_sub(boxScoreAll.gmDate, 1) == to_date(opptStandings.stDate_oppt)))
print(boxScoreAll.count())
print(boxWithStand.count())
boxWithStand.show()

2460
2460
+--------------------+------+-------+----------+-------+----------+--------+----------+-------+--------+--------+---------+-------+--------+----------+--------+--------+---------+-------+----------+---------+--------------------+---------+----+-------+-------+--------+---+-------+------+--------+------+--------+-------+--------+-------+--------+-------+--------+--------+-------+--------+--------+--------+-------+----------+---------+--------------+-------------+------+-----------+-----+-------+-------+------------+---------+---------+-----------+--------+--------+--------------------+--------------+---------+------------+------------+-------------+--------+------------+-----------+-------------+-----------+-------------+------------+-------------+------------+-------------+------------+-------------+-------------+------------+-------------+-------------+-------------+------------+---------------+--------------+-------------------+------------------+--------+----------------+-